# Librerías
Nota: Si faltan paquetes, instala con !pip install <paquete> en otra celda.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2
from scipy.spatial.distance import cdist
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats

from geopy.distance import great_circle  # O geodesic para mayor precisión
import folium  # Asumiendo instalado

import os
import pprint

import pmdarima as pm






# ¡¡¡ADVERTENCIA!!!

## Este codigo solo se corre la primera vez, posterior a eso el resultado se guardara en un txt. para recuperar el dataframe generado en caso de que se corra esta linea por segunda vez

### Configuración manual de las estaciones (agrega solo las que tengas)
station_files = {'05001': {'file': '05001.txt', 'lat': 27.92916667, 'lon': -100.575}, '05005': {'file': '05005.txt', 'lat': 26.84138889, 'lon': -100.7602778}, '05030': {'file': '05030.txt', 'lat': 27.51888889, 'lon': -100.6197222}, '05045': {'file': '05045.txt', 'lat': 27.61388889, 'lon': -100.725}, '05189': {'file': '05189.txt', 'lat': 27.59444444, 'lon': -100.7744444}, '05210': {'file': '05210.txt', 'lat': 26.81666667, 'lon': -100.7333333}, '19021': {'file': '19021.txt', 'lat': 26.49138889, 'lon': -100.0583333}, '19024': {'file': '19024.txt', 'lat': 27.23833333, 'lon': -100.1313889}, '19028': {'file': '19028.txt', 'lat': 27.07805556, 'lon': -100.4908333}, '19055': {'file': '19055.txt', 'lat': 27.42916667, 'lon': -100.3738889}, '19070': {'file': '19070.txt', 'lat': 27.03333333, 'lon': -100.5166667}, '19071': {'file': '19071.txt', 'lat': 26.49333333, 'lon': -99.52416667}, '19072': {'file': '19072.txt', 'lat': 27.28333333, 'lon': -100.0833333}, '19077': {'file': '19077.txt', 'lat': 26.65972222, 'lon': -99.98694444}, '19084': {'file': '19084.txt', 'lat': 26.5, 'lon': -100.0833333}, '19097': {'file': '19097.txt', 'lat': 27.70222222, 'lon': -99.75944444}, '19107': {'file': '19107.txt', 'lat': 27.53388889, 'lon': -99.97583333}, '19125': {'file': '19125.txt', 'lat': 27.43222222, 'lon': -99.80166667}, '19127': {'file': '19127.txt', 'lat': 26.88277778, 'lon': -99.82694444}, '19133': {'file': '19133.txt', 'lat': 26.50055556, 'lon': -100.1538889}, '19152': {'file': '19152.txt', 'lat': 27.06388889, 'lon': -100.0133333}, '19166': {'file': '19166.txt', 'lat': 27.17194444, 'lon': -100.4105556}, '19199': {'file': '19199.txt', 'lat': 26.56972222, 'lon': -99.79638889}, '28065': {'file': '28065.txt', 'lat': 27.48638889, 'lon': -99.50805556}, '28067': {'file': '28067.txt', 'lat': 26.58333333, 'lon': -99.18333333}, 'EU_Benavides2': {'file': 'EU_Benavides2.txt', 'lat': 27.596882, 'lon': -98.416414}, 'EU_Encinal1': {'file': 'EU_Encinal1.txt', 'lat': 27.977397, 'lon': -99.384733}, 'EU_Freer1': {'file': 'EU_Freer1.txt', 'lat': 27.872239, 'lon': -98.617604}, 'EU_Laredo1': {'file': 'EU_Laredo1.txt', 'lat': 27.533356, 'lon': -99.46666}, 'EU_Laredo2': {'file': 'EU_Laredo2.txt', 'lat': 27.568326, 'lon': -99.432792}}


In [ ]:

# 1. Inicializamos el diccionario que contendrá todos los datos.
station_files = {}

# 2. Obtenemos la lista de todos los archivos que terminan en .txt en la carpeta actual.
lista_archivos_txt = [archivo for archivo in os.listdir('.') if archivo.endswith('.txt')]

# 3. Recorremos cada archivo de la lista para procesarlo.
for nombre_archivo in lista_archivos_txt:
    lat = None
    lon = None

    try:
        # Usamos 'with open' para asegurar que el archivo se cierre automáticamente.
        with open(nombre_archivo, 'r', encoding='utf-8') as f:
            # Leemos cada línea del archivo.
            for linea in f:
                # 4. Buscamos la palabra 'LATITUD' en la línea.
                if 'LATITUD' in linea:
                    # Si la encuentra, extraemos el número.
                    # Pasos:
                    # 1. Divide la línea en dos por el ':' -> ' 27.48638889 ° '
                    # 2. .strip() quita espacios al inicio y final -> '27.48638889 °'
                    # 3. .split(' ')[0] divide por espacio y toma la primera parte -> '27.48638889'
                    valor_lat = linea.split(':')[1].strip().split(' ')[0]
                    lat = float(valor_lat)

                # 5. Hacemos lo mismo para la 'LONGITUD'.
                elif 'LONGITUD' in linea:
                    valor_lon = linea.split(':')[1].strip().split(' ')[0]
                    lon = float(valor_lon)

                # Si ya encontramos ambos valores, dejamos de leer el archivo.
                if lat is not None and lon is not None:
                    break

        # 6. Si se extrajeron ambos datos, los agregamos al diccionario.
        if lat is not None and lon is not None:
            # El 'id' de la estación será el nombre del archivo sin '.txt'.
            station_id = nombre_archivo.split('.')[0]
            station_files[station_id] = {
                'file': nombre_archivo,
                'lat': lat,
                'lon': lon
            }
        else:
            print(f"ADVERTENCIA: No se encontraron lat/lon en el archivo '{nombre_archivo}'.")

    except Exception as e:
        print(f"ERROR al procesar el archivo '{nombre_archivo}': {e}")

# 7. Finalmente, imprimimos el diccionario resultante con un formato legible.
print("\n--- Diccionario Final Generado ---")
pprint.pprint(station_files)

In [ ]:
print(station_files)

In [ ]:

map_center = [25.682500, -100.266944]  #--->  colocar coordenadas del lugar donde se centrara el mapa
m = folium.Map(location=map_center, zoom_start=10)
for sid, info in station_files.items():
    folium.Marker([info['lat'], info['lon']], popup=sid).add_to(m)

m.save('stations_map1.html')  # Abre en browser para interactividad

# ¡¡¡fin de la ADVERTENCIA!!!


# posterior a ejecuar la seccion anterior ya se puede limpiar los TXT para ejecutarlos


# Distancia geográfica

In [ ]:
# Esta funcios emplea la libreria Geopy para obtener un calculo 
# mucho mas preciso de distancias, considerando el metodo Haversine

def calculate_distance(lat1, lon1, lat2, lon2):
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    return great_circle(point1, point2).km  # Devuelve distancia en km usando Haversine internamente


In [ ]:
# esta funcion ya no se utiliza en el codigo,
# se emplea la libreria geopy en su lugar, 
# por mayor presicion al usar el metodo Haversine
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    esta primera seccion implementa la fórmula de Haversine, un método geométrico preciso para calcular la distancia
    más corta entre dos puntos sobre una esfera (en este caso, la Tierra aproximada como una esfera).
    Toma como entrada las coordenadas de dos puntos: lat1, lon1 (latitud y longitud del primer punto)
    y lat2, lon2 (latitud y longitud del segundo punto), todas en grados. Devuelve la distancia en
    kilómetros, lo que permite cuantificar la relación espacial entre estaciones
    """
    # Radio de la Tierra en km
    R = 6371.0  
    
    # Convierte a radianes la latitud y longitud de las cordenadas esfericas
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)
    
    # Calcula la diferencia entre los puntos de referencia
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Esta expresión representa la "haversine" de la diferencia angular entre
    # los puntos. Utiliza funciones trigonométricas (seno y coseno) para modelar
    # la geometría esférica. El término sin(dlat / 2)**2 y sin(dlon / 2)**2 
    # simplifica el cálculo evitando problemas de precisión cerca de los polos,
    #mientras que cos(lat1_rad) * cos(lat2_rad) ajusta por la curvatura latitudinal.
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Parsin, limpieza y ordenado de datos

In [ ]:


# Esta función actúa como el primer paso para procesar los datos crudos
# de las estaciones meteorológicas almacenados en archivos de texto (.txt).
# Su propósito principal es leer y transformar estos archivos en un formato
# estructurado (DataFrame de pandas) que pueda ser utilizado por las funciones
# posteriores de imputación, como idw_impute, mlr_impute y arima_impute

def parse_station_data(file_path, station_id, lat, lon):
    """
    Parsea un archivo .txt de estación meteorológica de CONAGUA, extrayendo FECHA y PRECIP.
    Maneja valores 'NULO' como NaN.
    """
    # Ubicacion donde guarda los datos generados
    data = []
    # revicion de los archivos y lectura de cada linea
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:  # Cambia a 'latin-1' si hay errores de encoding
        lines = f.readlines()
        parsing = False
        # Identificacion de la FECHA y PRECIP dentro del contenido 
        for line in lines:
            if 'FECHA' in line and 'PRECIP' in line:
                parsing = True
                continue
            # Si la identifica procede a clasificar si hay espacio y verificar si existen los dos datos den cada linea, fecha y precipitacion
            if parsing:
                parts = line.strip().split()
                if len(parts) >= 2:
                    # coloca a la fecha en la primera columna 
                    fecha = parts[0]
                    # coloca a la precipiracion en la segunda columnan si
                    # es nulo lo clasifica como Nan y lo integra como flotante, si da error lo pasa como Nan
                    precip = parts[1] if parts[1] != 'NULO' else np.nan
                    try:
                        precip = float(precip)
                    except ValueError:
                        precip = np.nan
                        #Agrega los elementos a la variable data, generada al principio 
                    data.append([fecha, precip, station_id, lat, lon])
                    # Los transforma en un dataframe
    df = pd.DataFrame(data, columns=['FECHA', 'PRECIP', 'station_id', 'lat', 'lon'])
    # configura la fecha a datatime, si da error borra la fecha y suregistro de los datos
    df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d', errors='coerce')  # Formato especificado para eliminar warnings
    return df.dropna(subset=['FECHA'])

# Inputacion de datos

## Inverse Distance Weighting (IDW)

Este método solo realiza la interpolación entre estaciones si existe valores dentro de la misma fecha, si este valor no existe no imputa y se pasa a analizar la siguiente fecha y datos existentes entre las estaciones para esa misma fecha, dentro de un rango límite de 50 km, lo que puede ser ajustado para considerar únicamente estaciones dentro de un rango aceptable

In [ ]:
def idw_impute(target_df, other_dfs, power=2, max_dist=50):
    """
    Imputa valores faltantes en PRECIP usando Inverse Distance Weighting (IDW).
    El objetivo es imputar los valores faltantes en target_df['PRECIP'] utilizando
    una media ponderada de los valores de precipitación de otras estaciones,
    donde el peso de cada estación se calcula como el inverso de la distancia
    elevada a power, restringido por max_dist. Esto refleja la idea de que las
    estaciones más cercanas tienen mayor influencia en la estimación.
    """
    # Se realiza una combinacion de los dataframe tanto del DF objetivo como el resto
    # concatena uniendo todos y los acomoda por fechas
    all_dfs = [target_df] + other_dfs
    combined = pd.concat(all_dfs).sort_values('FECHA')

    # Itera sobre las filas identificando el idx y los valores de cada fila (row)
    for idx, row in target_df[target_df['PRECIP'].isna()].iterrows():
        # Obtiene fecha y cordenadas de cada uno de los elemntos de la fila 
        fecha = row['FECHA']
        lat_t, lon_t = row['lat'], row['lon']

        # Realiza una busqueda de caondidatos que presenten nan en precipitacion y que tenga datos valodos en el resto de estaciones
        # si no existe algun valoe valodo en ninguna otra estacion el codigo pasa a analizar el siguiente renglon
        candidates = combined[(combined['FECHA'] == fecha) & (combined['station_id'] != row['station_id']) & combined['PRECIP'].notna()]
        if candidates.empty:
            continue
        # Se realiza el calculo de las sitancias desde la estacion a la estacion vecina con datos empleando la funcion creaqda al principio
        # en el apartado Distancia Geografica, si no hay candidatos en esa fecha salta al sigueinte valor para inputar
        dists = candidates.apply(lambda r: calculate_distance(lat_t, lon_t, r['lat'], r['lon']), axis=1)
        candidates = candidates[dists <= max_dist]
        if candidates.empty:
            continue
        # asignacion de posos para cada estacion en relacion a el inverso de la distancia elevada al factor power=2
        weights = 1 / (dists ** power)
        # Calcula la media ponderada de los valores de precipitación de las candidatas, normalizando por la suma de los pesos.
        imputed = np.sum(weights * candidates['PRECIP']) / np.sum(weights)
        #Asigna el valor inputado
        target_df.at[idx, 'PRECIP'] = max(0, imputed)

    return target_df

# Regresión Lineal Múltiple (MLR)

Integración en main_imputation: Actúa como un paso secundario tras idw_impute. Si quedan NaN después de la imputación espacial (e.g., en 2025-05-13 de '19185'), mlr_impute intenta completarlos usando correlaciones con otras estaciones.

In [ ]:
def mlr_impute(target_df, other_dfs):
    """
    Imputa usando Regresión Lineal Múltiple (MLR).
    utilizando los valores de precipitación de otras
    estaciones (other_dfs) como predictores. 
    Este método asume una relación lineal entre 
    las precipitaciones de la estación objetivo y
    las vecinas, ofreciendo una imputación basada en 
    correlaciones estadísticas cuando los datos 
    espaciales de idw_impute no son suficientes.
    """
    # Une los dataframe de otras estaciones al marge renombrando las columnas por id de sitio PRECIP_0, PRECIP_1, etc
    # estas colubnas las emplea como variables independientes, precervando fechas y agregando Nan en donde no se cuenta con datos
    merged = target_df.set_index('FECHA')
    for i, df in enumerate(other_dfs):
        merged = merged.join(df.set_index('FECHA')[['PRECIP']].rename(columns={'PRECIP': f'PRECIP_{i}'}), how='left')
    # Selecciona solo las filas donde no hay valores faltantes en ninguna columna, para entrenar el modelo.
    complete = merged.dropna()
    if len(complete) < 10:
        merged['PRECIP'] = merged['PRECIP'].fillna(merged['PRECIP'].mean())
        return merged.reset_index()
    # Define las variables independientes como todas las columnas excepto PRECIP
    X = complete.drop('PRECIP', axis=1)
    # Define la variable dependiente como PRECIP
    y = complete['PRECIP']
    # se realiza el entrenamiento del modelo empleando la libreria scikit-learn
    model = LinearRegression().fit(X, y)

    # Identifica las filas faltantes
    na_rows = merged[merged['PRECIP'].isna()]
    # Inputa el valor faltante identificado previamente
    if not na_rows.empty:
        X_na = na_rows.drop('PRECIP', axis=1).fillna(0)
        #Predice los valores de PRECIP usando el modelo entrenado.
        preds = model.predict(X_na)
        # asigna la prediccion a las posiciones faltantes 
        merged.loc[merged['PRECIP'].isna(), 'PRECIP'] = np.maximum(0, preds)
    
    return merged.reset_index()

# AutoRegressive Integrated Moving Average (ARIMA)

In [ ]:
def arima_impute_mejorado(df, gap_threshold=5):
    """
    Imputa gaps temporales largos usando un modelo SARIMA óptimo encontrado por auto_arima.
    
    Este método es una versión mejorada que:
    1. Utiliza pmdarima.auto_arima para encontrar el mejor orden (p,d,q).
    2. Incorpora estacionalidad (P,D,Q,m=365) para capturar patrones anuales de lluvia.
    3. Es más robusto al ajustarse dinámicamente a la estructura de cada serie temporal.
    """
    df = df.sort_values('FECHA').reset_index(drop=True)
    
    # La detección de gaps se mantiene igual, es muy eficiente.
    gaps = df['PRECIP'].isna().astype(int).groupby(df['PRECIP'].notna().cumsum()).sum()
    long_gaps = gaps[gaps >= gap_threshold]

    max_historico = df['PRECIP'].max() # Obtener el máximo conocido
    
    for gap_idx in long_gaps.index:
        # La lógica para encontrar el inicio y fin del gap también se mantiene.
        if gap_idx > 0:
            start_idx = df[df['PRECIP'].notna().cumsum() == gap_idx - 1].index[-1] + 1
        else:
            start_idx = 0
        
        gap_size = long_gaps[gap_idx]
        end_idx = start_idx + gap_size - 1

        before = df.loc[:start_idx-1, 'PRECIP'].dropna()
        after = df.loc[end_idx+1:, 'PRECIP'].dropna()

        # --- INICIO DE LA MEJORA ---
        
        # Se necesita suficiente data para un modelo estacional (idealmente > 2*m)
        # pero auto_arima puede manejar series más cortas. Aumentamos el umbral a 30.
        if len(before) > 30:
            try:
                # Usar auto_arima para encontrar el mejor modelo SARIMA
                sarima_model = pm.auto_arima(before,
                                             start_p=1, start_q=1,
                                             test='adf',       # test de estacionariedad
                                             max_p=3, max_q=3,  # órdenes máximos a probar
                                             m=365,            # ciclo estacional (diario)
                                             d=None,           # dejar que encuentre d
                                             seasonal=True,    # buscar modelo estacional
                                             start_P=0,
                                             D=1,              # D=1 suele funcionar para estacionalidad
                                             trace=False,      # no imprimir cada paso
                                             error_action='ignore',
                                             suppress_warnings=True,
                                             stepwise=True)    # búsqueda más rápida
                
                forecast = sarima_model.predict(n_periods=gap_size)
                forecast_clipped = np.clip(forecast.values, 0, max_historico)#
                df.loc[start_idx:end_idx, 'PRECIP'] = forecast_clipped #df.loc[start_idx:end_idx, 'PRECIP'] = np.maximum(0, forecast.values)

            except Exception as e: # Fallback si auto_arima falla
                print(f"AutoARIMA falló para gap {gap_idx}, usando interpolación lineal. Error: {e}")
                df['PRECIP'].interpolate(method='linear', limit_direction='both', inplace=True)

        # Pronóstico hacia atrás si no hay datos suficientes antes
        elif len(after) > 30:
            # La lógica es la misma, pero con la serie invertida (after[::-1])
            sarima_model = pm.auto_arima(after[::-1], seasonal=True, m=365, suppress_warnings=True, error_action='ignore', stepwise=True)
            forecast = sarima_model.predict(n_periods=gap_size)
            forecast_clipped = np.clip(forecast.values[::-1], 0, max_historico)
            df.loc[start_idx:end_idx, 'PRECIP'] = forecast_clipped #df.loc[start_idx:end_idx, 'PRECIP'] = np.maximum(0, forecast.values[::-1])
            
            
        
        # Fallback si no hay suficientes datos en ninguna dirección
        else:
            df.loc[start_idx:end_idx, 'PRECIP'] = df['PRECIP'].mean()

    # Interpolación final para cualquier NaN restante
    df['PRECIP'] = df['PRECIP'].interpolate(method='linear', limit_direction='both')
    return df

def arima_impute(df, gap_threshold=5, order=(1,1,1)):
    """
    Imputa gaps temporales largos usando ARIMA.
    implementa el modelo ARIMA (AutoRegressive 
    Integrated Moving Average) para imputar gaps
    largos de datos faltantes en df['PRECIP'], 
    enfocándose en patrones temporales. Es el 
    último recurso en el pipeline, utilizado 
    cuando los métodos espaciales (idw_impute)
    y estadísticos (mlr_impute) no completan 
    todos los NaN.
    """
    # se ordenan los datos por fechas 
    df = df.sort_values('FECHA')
    # deteccion de gaps, identifica los Nan clasificandolos con valores de 1
    # y valores validos en 0, los agrupa y calcula la longitud de cada GAP
    gaps = df['PRECIP'].isna().astype(int).groupby(df['PRECIP'].notna().cumsum()).sum()
    # Filtra gaps mayores o iguales a gap_threshold (default 5 días).
    long_gaps = gaps[gaps >= gap_threshold]

    # Procesamiento de Cada Gap Largo:
    for gap_idx in long_gaps.index:
        start = df[df['PRECIP'].notna().cumsum() == gap_idx - 1].index[-1] + 1 if gap_idx > 0 else 0
        end = start + long_gaps[gap_idx] - 1
        before = df.loc[:start-1, 'PRECIP'].dropna()
        after = df.loc[end+1:, 'PRECIP'].dropna()

        # imputa con ARIMA

        # Si hay al menos 10 valores antes, entrena
        # un modelo ARIMA con order=(1,1,1) (1 autorregresión,
        # 1 diferenciación, 1 media móvil) y predice hacia adelante.
        if len(before) > 10:
            model = ARIMA(before, order=order).fit()
            forecast = model.forecast(steps=long_gaps[gap_idx])
            df.loc[start:end, 'PRECIP'] = np.maximum(0, forecast)
        # Si no hay suficientes datos antes pero sí después,
        # entrena un modelo con la serie invertida y predice hacia atrás.    
        elif len(after) > 10:
            model = ARIMA(after[::-1], order=order).fit()
            forecast = model.forecast(steps=long_gaps[gap_idx])[::-1]
            df.loc[start:end, 'PRECIP'] = np.maximum(0, forecast)
        # Si no hay suficientes datos en ninguna dirección,
        # usa la media de df['PRECIP'].mean() como fallback.
        else:
            df.loc[start:end, 'PRECIP'] = df['PRECIP'].mean()
    #Asigna las predicciones, asegurando valores no negativos.
    df['PRECIP'] = df['PRECIP'].interpolate(method='linear', limit_direction='both')
    return df

# Iteración sobre las estaciones + orden jerárquico

El objetivo es iterar sobre todas las estaciones, leer sus datos crudos, aplicar un enfoque jerárquico de imputación (IDW → MLR → ARIMA), y devolver un diccionario (imputed) con los DataFrames resultantes, donde los valores faltantes de precipitación (PRECIP) han sido estimados.

In [ ]:
def main_imputation(station_files):
    """
    Función principal: Procesa múltiples archivos de estaciones, aplica IDW/MLR y ARIMA sobre rango completo de fechas.
    """
    dfs = {}
    all_dates = set()  # Para recopilar todas las fechas únicas
    
    # Leer datos y recopilar fechas
    for sid, info in station_files.items():
        df = parse_station_data(info['file'], sid, info['lat'], info['lon'])
        dfs[sid] = df
        all_dates.update(df['FECHA'])
    
    # Rango global: min a max fecha, frecuencia diaria
    if all_dates:
        min_date = min(all_dates)
        max_date = max(all_dates)
        full_date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    else:
        return {}  # No hay datos
    
    imputed = {}
    for sid in dfs:
        target = dfs[sid].set_index('FECHA').reindex(full_date_range).reset_index().rename(columns={'index': 'FECHA'})
        target['station_id'] = sid  # Asignar station_id como constante para todas las filas
        target['lat'] = station_files[sid]['lat']
        target['lon'] = station_files[sid]['lon']
        others = [dfs[s].set_index('FECHA').reindex(full_date_range).reset_index().rename(columns={'index': 'FECHA'}) for s in dfs if s != sid]
        for i, o in enumerate(others):
            o['station_id'] = dfs[list(dfs.keys())[i]]['station_id'].iloc[0]  # Asignar ID correcto con longitud adecuada
        
        # Primero IDW espacial
        target = idw_impute(target, others)
        
        # Luego MLR si quedan NaNs
        if target['PRECIP'].isna().any():
            target = mlr_impute(target, others)
        
        # Finalmente ARIMA para gaps temporales
        if target['PRECIP'].isna().any():
            target = arima_impute_mejorado(target)
        
        imputed[sid] = target
    
    return imputed

Realizaremos la importacion y posterior limpieza de los archivos

In [ ]:
# Ejecutar la imputación
results = main_imputation(station_files)

In [ ]:
# Ejemplo para visualizar resultados de una estación específica (ejemplo: 19185)
#print("Resultados imputados para estación 19052:")
#print(results['19052'].head(10))  # Muestra las primeras 10 filas

# Exportar a CSV para análisis posterior
for sid, df in results.items():
    df.to_csv(f'{sid}_imputado.csv', index=False)
    print(f"Archivo exportado: {sid}_imputado.csv")


# Explicación del Gráfico

Líneas: La línea azul muestra los valores originales de PRECIP (con NaN donde había 'NULO'), mientras que la roja muestra los valores imputados.
Puntos Verdes: Marcadores que indican dónde se imputaron valores, destacando visualmente los reemplazos de 'NULO'.
Ejes: El eje X representa las fechas, y el Y la precipitación en mm. La rotación de etiquetas en X mejora la legibilidad.
Personalización: Título, leyenda y rejilla facilitan la interpretación.

# Atención  apartir de esta seccion es necesario identificar la estacion de trabajo siga las indicaciones marcadas en #  ---> prueba con estacion Monterrey 19049  25.682500, -100.266944

In [ ]:
station_id = '19200' # ---> Coloque la estacion sobre la que se centraran los estadsiticos y analisis 19049


In [ ]:
map_center = [25.682500, -100.266944]  #--->  colocar coordenadas del lugar donde se centrara el mapa
m = folium.Map(location=map_center, zoom_start=10)


for sid, info in station_files.items():
    folium.Marker([info['lat'], info['lon']], popup=sid).add_to(m)

# ---> colocar clave de las estaciones point1 = mas alejada  point2 = la estacion de estudio 
point1 = (station_files['19169']['lat'], station_files['19169']['lon'])
point2 = (station_files['19200']['lat'], station_files['19200']['lon'])
dist = calculate_distance(*point1, *point2)  # O con Geopy
folium.PolyLine([point1, point2], color="red", weight=2.5, opacity=1, popup=f"Dist: {dist:.2f} km").add_to(m)

m.save('stations_map.html')  # Abre en browser para interactividad

# Atención

### Grafico con rango limitado a gusto

In [ ]:
df_original = parse_station_data(station_files[station_id]['file'], station_id, station_files[station_id]['lat'], station_files[station_id]['lon'])
df_imputed = results[station_id]


df_original = df_original.set_index('FECHA')
df_imputed = df_imputed.set_index('FECHA')
combined_df = df_original.join(df_imputed['PRECIP'].rename('PRECIP_imputed'), how='outer')

# Filtrar solo el año 2025
combined_df_2025 = combined_df.loc['2009-01-01':'2011-12-31']  # ---> ajusta rango de fechas de observacion en el grafico


plt.figure(figsize=(18, 6))
plt.plot(combined_df_2025.index, combined_df_2025['PRECIP'], 'b-', label='Original (NULO como NaN)', alpha=0.5, marker='o')
plt.plot(combined_df_2025.index, combined_df_2025['PRECIP_imputed'], 'r-', label='Imputado', alpha=0.7, marker='o')

plt.title(f'Comparación de Precipitación Original vs Imputada - Estación {station_id} (2025)')
plt.xlabel('Fecha')
plt.ylabel('Precipitación (mm)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

nulos_idx = combined_df_2025['PRECIP'].isna()
plt.scatter(combined_df_2025.index[nulos_idx], combined_df_2025['PRECIP_imputed'][nulos_idx], color='green', label='Nulos Imputados', zorder=5)

plt.legend()
plt.show()

# Atención

### grafico con rango completo 

In [ ]:
df_original = parse_station_data(station_files[station_id]['file'], station_id, station_files[station_id]['lat'], station_files[station_id]['lon'])
df_imputed = results[station_id]

df_original = df_original.set_index('FECHA')
df_imputed = df_imputed.set_index('FECHA')
combined_df = df_original.join(df_imputed['PRECIP'].rename('PRECIP_imputed'), how='outer')

plt.figure(figsize=(12, 6))
plt.plot(combined_df.index, combined_df['PRECIP'], 'b-', label='Original (NULO como NaN)', alpha=0.5, marker='o')
plt.plot(combined_df.index, combined_df['PRECIP_imputed'], 'r-', label='Imputado', alpha=0.7, marker='o')

# Personalizar el gráfico
plt.title(f'Comparación de Precipitación Original vs Imputada - Estación {station_id}')
plt.xlabel('Fecha')
plt.ylabel('Precipitación (mm)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

nulos_idx = combined_df['PRECIP'].isna()
plt.scatter(combined_df.index[nulos_idx], combined_df['PRECIP_imputed'][nulos_idx], color='green', label='Nulos Imputados', zorder=5)

plt.legend()
plt.show()

### Grafico con Rango Completo de Datos Reales vs Nulos Actualizados

In [ ]:
df_original = parse_station_data(station_files[station_id]['file'], station_id, station_files[station_id]['lat'], station_files[station_id]['lon'])
df_imputed = results[station_id]

# Asegurar que las fechas estén alineadas
df_original = df_original.set_index('FECHA')
df_imputed = df_imputed.set_index('FECHA')
combined_df = df_original.join(df_imputed['PRECIP'].rename('PRECIP_imputed'), how='outer')

# Filtrar solo el año 2025
combined_df_2025 = combined_df.loc['2020-01-01':'2021-12-31']

# Separar datos reales (no nulos originales) y nulos actualizados (imputados donde había NaN)
real_data = combined_df_2025['PRECIP'].dropna()  # Valores originales no nulos
imputed_data = combined_df_2025[combined_df_2025['PRECIP'].isna()]['PRECIP_imputed']  # Valores imputados donde había nulos

# Crear el gráfico
plt.figure(figsize=(18, 6))

# Graficar datos reales
plt.plot(real_data.index, real_data, 'bo-', label='Datos Reales', alpha=0.7, marker='o')

# Graficar nulos actualizados
plt.plot(imputed_data.index, imputed_data, 'ro-', label='Nulos Actualizados', alpha=0.7, marker='o')

# Personalizar el gráfico
plt.title(f'Datos Reales vs Nulos Actualizados - Estación {station_id} (2025)')
plt.xlabel('Fecha')
plt.ylabel('Precipitación (mm)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Añadir una línea de referencia para cero
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)

plt.legend()
plt.show()

# Opcional: Guardar el gráfico
# plt.savefig(f'real_vs_imputed_{station_id}_2025.png')

## Evaluacion de distribuciones de ajuste

In [ ]:
def evaluate_best_distribution(data, distributions=None):
    """
    Evalúa la mejor distribución que se ajusta a los datos no nulos.
    Retorna el nombre de la mejor distribución y sus parámetros.
    """
    if distributions is None:
        distributions = [stats.gamma, stats.lognorm, stats.weibull_min, stats.expon]
    
    data = data.dropna().values  # Solo datos no nulos
    data = data[data > 0]  # Precipitación >0 para distribuciones positivas
    
    if len(data) < 3:
        return "No suficiente datos para ajustar", None
    
    results = {}
    for dist in distributions:
        params = dist.fit(data)
        ks_stat, ks_pvalue = stats.kstest(data, dist.cdf, args=params)
        results[dist.name] = (ks_stat, ks_pvalue, params)
    
    # Mejor distribución: Mayor p-value en KS-test (mejor ajuste)
    best_dist_name = max(results, key=lambda k: results[k][1])
    best_ks_stat, best_ks_pvalue, best_params = results[best_dist_name]
    
    return best_dist_name, best_params

df_original = parse_station_data(station_files[station_id]['file'], station_id, station_files[station_id]['lat'], station_files[station_id]['lon'])
best_dist_orig, params_orig = evaluate_best_distribution(df_original['PRECIP'])
print(f"Mejor distribución para originales de {station_id}: {best_dist_orig} con parámetros {params_orig}")

In [ ]:

df_imputed = results[station_id]

### Comparacion estadistica DR vs DR+Imputados

In [ ]:
def calculate_statistics(data, label):
    """
    Calcula estadísticos descriptivos.
    """
    stats_dict = {
        'Media': data.mean(),
        'Mediana': data.median(),
        'Desviación Estándar': data.std(),
        'Skewness': data.skew(),
        'Kurtosis': data.kurtosis(),
        'Mínimo': data.min(),
        'Máximo': data.max()
    }
    print(f"Estadísticos para {label}:")
    for key, value in stats_dict.items():
        print(f"{key}: {value:.4f}")
    return stats_dict

def compare_with_ks_rmse_mae(orig_data, imp_data):
    """
    Compara distribuciones con KS-test y calcula RMSE/MAE.
    """
    # Usar solo valores no nulos para KS-test
    orig_values = orig_data.dropna().values
    imp_values = imp_data.dropna().values  # Usar solo datos no nulos para consistencia
    
    # KS-test para distribuciones
    ks_stat, ks_pvalue = stats.kstest(orig_values, imp_values)
    print(f"KS-test (distribuciones): stat={ks_stat:.4f}, p-value={ks_pvalue:.4f}")
    if ks_pvalue > 0.05:
        print("Distribuciones consistentes (p>0.05)")
    else:
        print("Distribuciones difieren significativamente")
    
    # RMSE y MAE para comparación punto a punto usando índices comunes
    orig_df = orig_data.to_frame().dropna(subset=['PRECIP'])
    imp_df = imp_data.to_frame().dropna(subset=['PRECIP'])
    common_dates = orig_df.index.intersection(imp_df.index)
    
    if len(common_dates) > 0:
        orig_common = orig_df.loc[common_dates, 'PRECIP']
        imp_common = imp_df.loc[common_dates, 'PRECIP']
        rmse = np.sqrt(np.mean((orig_common - imp_common)**2))
        mae = np.mean(np.abs(orig_common - imp_common))
        print(f"RMSE (error cuadrático): {rmse:.4f}")
        print(f"MAE (error absoluto): {mae:.4f}")
    else:
        print("No hay suficientes fechas comunes para calcular RMSE/MAE")

# Estadísticos originales
orig_stats = calculate_statistics(df_original['PRECIP'], "Originales")

# Estadísticos imputados
imputed_stats = calculate_statistics(df_imputed['PRECIP'], "Imputados")

# Comparación integral
compare_with_ks_rmse_mae(df_original.set_index('FECHA')['PRECIP'], df_imputed.set_index('FECHA')['PRECIP'])

# Diferencia en dispersión
dispersion_diff = abs(orig_stats['Desviación Estándar'] - imputed_stats['Desviación Estándar'])
print(f"Diferencia en dispersión (std): {dispersion_diff:.4f}")

In [ ]:
def plot_distribution_fit_comparison(orig_data, imp_data, dist_name_orig, params_orig, dist_name_imp, params_imp):
    """
    Grafica PDF y Q-Q plots comparativos para originales vs imputados.
    """
    orig_data = orig_data.dropna().values
    imp_data = imp_data.values
    
    x_orig = np.linspace(orig_data.min(), orig_data.max(), 100)
    x_imp = np.linspace(imp_data.min(), imp_data.max(), 100)
    
    dist_orig = getattr(stats, dist_name_orig)
    dist_imp = getattr(stats, dist_name_imp)
    pdf_orig = dist_orig.pdf(x_orig, *params_orig)
    pdf_imp = dist_imp.pdf(x_imp, *params_imp)
    
    plt.figure(figsize=(15, 6))
    
    # Histograma y PDF comparativos
    plt.subplot(1, 3, 1)
    plt.hist(orig_data, bins=30, density=True, alpha=0.5, label='Original', color='blue')
    plt.hist(imp_data, bins=30, density=True, alpha=0.5, label='Imputado', color='red')
    plt.plot(x_orig, pdf_orig, 'b-', label='PDF Original')
    plt.plot(x_imp, pdf_imp, 'r-', label='PDF Imputado')
    plt.title('Comparación de Distribuciones')
    plt.xlabel('PRECIP (mm)')
    plt.ylabel('Densidad')
    plt.legend()
    
    # Q-Q plot originales
    plt.subplot(1, 3, 2)
    stats.probplot(orig_data, dist=dist_name_orig, sparams=params_orig, plot=plt)
    plt.title('Q-Q Plot Originales')
    
    # Q-Q plot imputados
    plt.subplot(1, 3, 3)
    stats.probplot(imp_data, dist=dist_name_imp, sparams=params_imp, plot=plt)
    plt.title('Q-Q Plot Imputados')
    
    plt.tight_layout()
    plt.show()

# Evaluar distribuciones
best_dist_orig, params_orig = evaluate_best_distribution(df_original['PRECIP'])
best_dist_imp, params_imp = evaluate_best_distribution(df_imputed['PRECIP'])

print(f"Distribución original: {best_dist_orig}")
print(f"Distribución imputada: {best_dist_imp}")

# Gráfico comparativo
plot_distribution_fit_comparison(df_original['PRECIP'], df_imputed['PRECIP'], best_dist_orig, params_orig, best_dist_imp, params_imp)

# Consistencia: Si tipos coinciden y KS p>0.05, son consistentes
if best_dist_orig == best_dist_imp:
    print("Tipos de distribución coinciden.")
else:
    print("Tipos difieren; verificar con KS.")

In [ ]:
def hydrological_validation(orig_df, imp_df):
    """
    Validación específica para hidrología: Correlación, frecuencias mensuales y ajuste anual.
    """
    orig_df = orig_df.set_index('FECHA')
    imp_df = imp_df.set_index('FECHA')
    
    # Correlación en fechas comunes
    common_dates = orig_df.dropna().index.intersection(imp_df.index)
    corr = orig_df.loc[common_dates, 'PRECIP'].corr(imp_df.loc[common_dates, 'PRECIP'])
    print(f"Correlación Pearson (datos comunes): {corr:.4f}")
    
    # Frecuencias mensuales (como en manuales CONAGUA)
    orig_monthly = orig_df['PRECIP'].resample('ME').sum().dropna()
    imp_monthly = imp_df['PRECIP'].resample('ME').sum().dropna()
    monthly_corr = orig_monthly.corr(imp_monthly)
    print(f"Correlación en precipitación mensual: {monthly_corr:.4f}")
    
    # Ajuste anual (si hay datos anuales)
    if len(orig_monthly) >= 12:
        orig_annual = orig_monthly.resample('YE').sum()
        imp_annual = imp_monthly.resample('YE').sum()
        annual_rmse = np.sqrt(np.mean((orig_annual - imp_annual)**2))
        print(f"RMSE en precipitación anual: {annual_rmse:.4f}")

# Aplicar validación
hydrological_validation(df_original, df_imputed)

# Atención

In [ ]:
def generate_monthly_max_precip(df, station_id):
    """
    Genera un DataFrame con precipitación mensual máxima, pivoteado por año y meses.
    """
    df = df.copy()  
    df['Año'] = df['FECHA'].dt.year
    df['Mes'] = df['FECHA'].dt.month_name(locale='es_ES')  
    
    monthly_max = df.groupby(['Año', 'Mes'])['PRECIP'].max().reset_index()
    
    month_order = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    pivot_df = monthly_max.pivot(index='Año', columns='Mes', values='PRECIP').fillna(0)  # Rellena con 0 si no hay datos
    
    pivot_df = pivot_df.reindex(columns=[m.capitalize() for m in month_order])
    
    return pivot_df

df_imputed = results[station_id]  

monthly_max_df = generate_monthly_max_precip(df_imputed, station_id)
print("DataFrame de Precipitación Mensual Máxima (mm):")
print(monthly_max_df.to_string())

# Exportar a CSV si es necesario
monthly_max_df.to_csv(f'monthly_max_precip_{station_id}.csv')

# Atención

In [ ]:
# Ejecutar la imputación con rango completo
results_full = results

df_imputed_full = results_full[station_id]
print(f"DataFrame imputado con rango completo para {station_id}:")
print(df_imputed_full.head())  # Muestra primeras filas

# Opcional: Guardar
df_imputed_full.to_csv(f'{station_id}_imputado_full_range.csv', index=False)

determinar la distribucion de cada estacion y compararlas entre si calculando la desviacion estandar, R2, varianza y observar como se compartan entre si, escojer la que tenga mejor correlacion temporal y emplearla como peso para el calculo de las imputacion por los metodos antes empleados IDW ARIMA LR